<h1>Computational Theory Problems</h1>

In [ ]:
import numpy

<h1>Problem 1: Binary Words and Operations</h1>

<h1>Problem 2: Fractional Parts of Cube Roots</h1>

<h1>Problem 3: Padding</h1>

<h1>Problem 4: Hashes</h1>

<h1>Problem 5: Passwords</h1>